In [1]:
import tide_constituents as tc
import noaa_coops as nc
import pandas as pd
import numpy as np
import datetime
import tappy
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
pd.plotting.register_matplotlib_converters()

In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

In [ ]:
start = '20180201'
end = '20180228'
interval = 1

In [ ]:
pr = tc.get_tides('20180101', '20180120', -88.2, 30.4)

In [ ]:
noaa_predict = tc.get_tides(start, end, -88.2, 30.4)

In [ ]:
start = pd.to_datetime(start)
end = pd.to_datetime(end)
d = start
w, t, p, r = [], [], [], []

while d < end:
    start_ = d
    end_ = start_ + pd.DateOffset(interval)
    end_ = end_ if end_ < end else end
    water_level = tc.get_water_levels(start_.strftime('%Y%m%d'),
                                      end_.strftime('%Y%m%d'),
                                      -88.2, 30.4)
    tide = tc.tide_constituents(water_level)
    water_level = water_level.water_level.astype('float')
    prediction = 0.0 if 'Z0' not in list(tide.speed_dict.keys()) else tide.speed_dict['Z0']
    prediction += tc.sum_signals(tide.key_list, tide.dates, tide.speed_dict, tide.r, tide.phase)
    residual = water_level - prediction
    w.append(water_level)
    t.append(tide)
    p.append(prediction)
    r.append(residual)
    d = end_

In [ ]:
df_cons = pd.DataFrame({'amp': list(t[0].r.values()),
                        'phase': list(t[0].phase.values()),
                        'speed': [t[0].speed_dict[i]['speed'] for i in list(t[0].speed_dict.keys()) if not i == 'P1'],
                        'VAU': [t[0].speed_dict[i]['VAU'] for i in list(t[0].speed_dict.keys()) if not i == 'P1']})

In [ ]:
df_cons.to_csv('constituents_claw.csv', float_format='%3.10f', index=False, header=None)

In [ ]:
df_FF = pd.DataFrame([t[0].speed_dict[i]['FF'] for i in list(t[0].speed_dict.keys()) if not i == 'P1']).T

In [ ]:
df_FF.to_csv('constituents_FF_claw.csv', index=False, header=None)

In [ ]:
water_level = pd.concat(w).to_frame()
water_level.columns = ['observation']
water_level['prediction'] = np.hstack(p)
water_level['residual'] = np.hstack(r)
#water_level = water_level[['water_level', 'prediction', 'residual']]
#water_level.columns = ['observation', 'prediction', 'residual']
water_level = water_level[['observation', 'prediction']]

In [ ]:
ax = water_level.plot()
noaa_predict.plot(ax=ax)
ax.legend(['observation', 'prediction', 'NOAA prediction'], loc='best')
ax.set_xlabel('');

In [ ]:
year = '2018'

In [ ]:
start = year + '0101'
end = year + '1231'
data = tc.get_water_levels(start, end, -88.2, 30.4)

In [ ]:
pr.head()

In [ ]:
wl = data.water_level.copy()
grouped = wl.groupby(pd.Grouper(freq='M'))

def f(group):
        return pd.DataFrame({'original': group, 'demeaned': group - group.mean()})

wl_demeaned = grouped.apply(f)

In [ ]:
ax = wl_demeaned.plot()
ax.set_xlabel('');

In [ ]:
min_month = wl_demeaned.rolling(30).min().groupby(pd.Grouper(freq='M')).last()
max_month = wl_demeaned.rolling(30).max().groupby(pd.Grouper(freq='M')).last()
monthly_minmax = min_month.copy()
monthly_minmax['high'] = max_month['demeaned']
monthly_minmax = monthly_minmax[['demeaned', 'high']]
monthly_minmax.columns = ['low', 'high']
monthly_minmax['range'] = monthly_minmax.high - monthly_minmax.low
ranked = monthly_minmax.sort_values('range')
ranked